In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import ast
import numpy as np

2023-06-28 12:51:00.401608: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-28 12:51:00.425388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 12:51:00.754552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Step 1: Load and split the dataset
csv_file_path = 'balenced_formatted_torch_data.csv'
api_data = pd.read_csv(csv_file_path)

In [3]:
X = api_data[['API Name', 'Parameters']]
y = api_data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Step 2: Extract numerical values from the 'Parameters' column
def extract_numerical_values(param_string):
    try:
        # Modify this function to extract numerical values based on your specific string format
        param_list = ast.literal_eval(param_string)
        numerical_values = [float(value) if value != 'nan' else np.nan for value in param_list]
    except (SyntaxError, ValueError):
        # Handle malformed values or strings that cannot be evaluated
        numerical_values = [np.nan]  # Replace malformed values with NaN

    return numerical_values

In [5]:
X_train['Parameters'] = X_train['Parameters'].apply(extract_numerical_values)
X_test['Parameters'] = X_test['Parameters'].apply(extract_numerical_values)

In [6]:
# Step 3: Pad sequences to ensure they have the same length
max_seq_length = max(max(X_train['Parameters'].apply(len)), max(X_test['Parameters'].apply(len)))

X_train['Parameters'] = X_train['Parameters'].apply(lambda x: x + [0.0] * (max_seq_length - len(x)))
X_test['Parameters'] = X_test['Parameters'].apply(lambda x: x + [0.0] * (max_seq_length - len(x)))

In [7]:
# Step 3: Encode categorical features
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[['API Name']])
X_test_encoded = encoder.transform(X_test[['API Name']])

/home/baidu/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
# Step 5: Prepare input features
X_train_input = tf.concat([X_train_encoded, tf.convert_to_tensor(X_train['Parameters'].tolist())], axis=1)
X_test_input = tf.concat([X_test_encoded, tf.convert_to_tensor(X_test['Parameters'].tolist())], axis=1)

2023-06-28 12:51:01.304471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-28 12:51:01.334505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-28 12:51:01.334644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
# Step 4: Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_input.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [10]:
# Step 5: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
# Step 6: Train the model
model.fit(X_train_input, y_train, batch_size=32, epochs=10)

Epoch 1/10


2023-06-28 12:51:02.021777: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "/home/baidu/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/baidu/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/baidu/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/baidu/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/baidu/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/baidu/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/baidu/anaconda3/lib/python3.10/asyncio/base_events

In [ ]:
# Step 7: Evaluate the model
loss, accuracy = model.evaluate(X_test_input, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

In [ ]:
# Step 8: Make predictions
'''
new_test_case = pd.DataFrame({'API Name': ['your_api_name'], 'Parameters': ['your_parameters']})
new_test_case_encoded = encoder.transform(new_test_case[['API Name']])
new_test_case_input = tf.concat([new_test_case_encoded, new_test_case['Parameters']], axis=1)
prediction = model.predict(new_test_case_input)
print(f'Probability of triggering a bug: {prediction[0][0]}')

'''
